In [2]:
import os # OS e.g directory structure
import numpy as np # linear algebra
import scipy as sc  # scientific computing
import pandas as pd # data processing, file I/O
import seaborn as sns  # visualization
import matplotlib.pyplot as plt # visualization
import warnings
warnings.filterwarnings("ignore")

In [3]:

import sys

from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [4]:

! ls -la
! head -n 3 library-collection-inventory.csv

total 22983768
drwxr-xr-x  5 macbookpro  staff          160 16 Mai 09:02 .
drwxr-xr-x  6 macbookpro  staff          192  6 Mai 14:27 ..
drwxr-xr-x  3 macbookpro  staff           96  7 Mai 00:31 .ipynb_checkpoints
-rw-r--r--  1 macbookpro  staff        17547 16 Mai 09:02 TP_BIG_DATA.ipynb
-rw-rw-r--@ 1 macbookpro  staff  11764863851  1 Dez  2019 library-collection-inventory.csv
BibNum,Title,Author,ISBN,PublicationYear,Publisher,Subjects,ItemType,ItemCollection,FloatingItem,ItemLocation,ReportDate,ItemCount
3011076,"A tale of two friends / adapted by Ellie O'Ryan ; illustrated by Tom Caulfield, Frederick Gardner, Megan Petasky, and Allen Tam.","O'Ryan, Ellie","1481425730, 1481425749, 9781481425735, 9781481425742",2014.,"Simon Spotlight,","Musicians Fiction, Bullfighters Fiction, Best friends Fiction, Friendship Fiction, Adventure and adventurers Fiction",jcbk,ncrdr,Floating,qna,2017-09-01T00:00:00.000,1
2248846,"Naruto. Vol. 1, Uzumaki Naruto / story and art by Masashi Kishimoto ; [Engli

In [5]:
from pyspark.sql.types import StructField, StructType, StringType, IntegerType, TimestampType, DoubleType

fire_schema = StructType([StructField("BibNum", IntegerType(),True),
                             StructField("Title", StringType(),True),
                             StructField("Author", StringType(),True),
                             StructField("ISBN", IntegerType(),True),
                             StructField("PublicationYear", IntegerType(),True),
                             StructField("Publisher", StringType(),True),
                             StructField("Subjects", StringType(),True),
                             StructField("ItemType", StringType(),True),
                             StructField("ItemCollection", StringType(),True),
                             StructField("FloatingItem", StringType(),True),
                             StructField("ItemLocation", StringType(),True),
                             StructField("ReportDate", TimestampType(),True),
                             StructField("ItemCount", IntegerType(),True)
                            ])


In [6]:


df = spark.read.csv('library-collection-inventory.csv',header=True, schema=fire_schema, sep=",")



### Exploratory data analysis

In [7]:
df.printSchema()
cols = df.columns
df.count()

root
 |-- BibNum: integer (nullable = true)
 |-- Title: string (nullable = true)
 |-- Author: string (nullable = true)
 |-- ISBN: integer (nullable = true)
 |-- PublicationYear: integer (nullable = true)
 |-- Publisher: string (nullable = true)
 |-- Subjects: string (nullable = true)
 |-- ItemType: string (nullable = true)
 |-- ItemCollection: string (nullable = true)
 |-- FloatingItem: string (nullable = true)
 |-- ItemLocation: string (nullable = true)
 |-- ReportDate: timestamp (nullable = true)
 |-- ItemCount: integer (nullable = true)



35531308

# Data cleaning




In [8]:
df_1 = df.dropna(subset=["Author","Publisher"])

In [12]:
df_1.select("Author","ItemLocation","ISBN","PublicationYear","Title","ReportDate").count()

29943970

In [13]:
max_row =10000

df_clean = df_1.limit(max_row)

df_clean.count()

10000

In [26]:
df_clean.show()

df_clean.createOrReplaceTempView("library")

+-------+--------------------+--------------------+----------+---------------+--------------------+--------------------+--------+--------------+------------+------------+-------------------+---------+
| BibNum|               Title|              Author|      ISBN|PublicationYear|           Publisher|            Subjects|ItemType|ItemCollection|FloatingItem|ItemLocation|         ReportDate|ItemCount|
+-------+--------------------+--------------------+----------+---------------+--------------------+--------------------+--------+--------------+------------+------------+-------------------+---------+
|1898806|Paul Rand / Steve...|      Heller, Steven| 714837989|           1999|            Phaidon,|Rand Paul 1914 19...|    arbk|          cs8r|          NA|         cen|2017-09-01 00:00:00|        1|
| 397637|The case against ...|  Priestland, Gerald|   2151421|           1984|            Collins,|           God Proof|    acbk|          canf|          NA|         cen|2017-09-01 00:00:00|      

In [ ]:
sql = spark.sql("""
                 select Author, ISBN
                from library 
                where 
               
               """
               )

In [20]:
col_to_drop = ['ItemCount','FloatingItem','BibNum','ReportDate']

In [23]:
cols_feactures = [c for c in cols if c not in col_to_drop]
col_interesting = ['ISBN','PublicationYear']

In [ ]:
df_clean.write.format("parquet").mode("overwrite")\
                .save("library-file.parquet")

In [ ]:
! ls -la

In [25]:
from pyspark.ml.stat import Correlation
from pyspark.ml.feature import VectorAssembler

# Correlation requires vectors so prior we convert to vector column

vector_col = "corr_features"
assembler = VectorAssembler(inputCols=col_interesting, outputCol=vector_col)
df_vector = assembler.transform(df_clean).select(vector_col)

# get correlation matrix

matrix = Correlation.corr(df_vector, vector_col).collect()[0][0]
corrmatrix = matrix.toArray().tolist()
# corrmatrix
corrmatrix

[[1.0, 0.12758359452724763], [0.12758359452724763, 1.0]]